## 1. 인물모드 직접 해 보기
-최소 3장 이상의 인물모드 사진을 만들어
-닌, 귀여운 고양이에 대한 아웃포커싱 사진도 만들어 볼 수 있을 것입니다. 시맨틱 세그멘테이션 스텝에서 힌트를 찾아
-배경을 blur하는 인물모드 사진이 아니라 배경사진을 다른 이미지로 교체하는 크로마키 배경합성을 시도해 볼 수도

## 2. 사진에서 문제점 찾기


 ####  Segmentation의 윤곽선이 드러난다.
 
![img](face_focusing.png)

![img](cat2_result.png)

![img](cat1_result.png)


## 3. 해결 방법을 제안해 보기

 - 세그멘테이션의 한계
Semantic segmentation의 부정확성이 여러가지 문제를 발생시키는 주요 원인입니다. 피사계심도를 이용한 보케(아웃포커싱) 효과는 말 그대로 심도를 표현하기 때문에 초점이 잡힌 거리를 광학적으로 아주 섬세하게 구별(segmentation) 하지만 이를 따라한 semantic segmentation 모듈은 정확도가 1.00 이 되지 않는 한 완벽히 구현하기는 힘듭니다.

 - 피사계 심도 이해하기
우선 피사계심도의 개념부터 명확히 이해해 봅시다.

참고자료 : https://ggyul.tistory.com/12

 - 3D Depth Camera 활용하기
카메라 2개를 통해 생성한 3d 영상으로 인물모드를 더 정확하게 만들 수 있을까요? 우리는 카메라 1대가 찍은 영상에서 semantic segmentation을 시도하였고, 그 결과가 정확하지 않았습니다. 요즘 스마트폰의 카메라는 렌즈가 2개 달려있지요? 왜 굳이 그렇게까지 하는지 아래 링크를 보면서 이해해 봅시다.

참고자료 : https://m.blog.naver.com/panoptics/221336152952

 - 깊이 영상(Depth image) 활용하기
하지만 꼭 카메라가 2개여야 할까요? 아래 이미지는 Struct2Depth 라는 기법을 소개한 Google Brain의 논문에 있는 이미지입니다. 세번째 컬럼이 보다 명확하게 depth에 따른 물체인식을 보여 주는데, 이것은 LiDAR가 없이도 아주 정확한 segmentation을 동반한 depth sensor가 가능함을 보여줍니다.


In [6]:
import cv2
import numpy as np
import os
from glob import glob
from os.path import join
import tarfile
import urllib

from matplotlib import pyplot as plt
import tensorflow as tf

import os

class DeepLabModel(object):
    INPUT_TENSOR_NAME = 'ImageTensor:0'
    OUTPUT_TENSOR_NAME = 'SemanticPredictions:0'
    INPUT_SIZE = 513
    FROZEN_GRAPH_NAME = 'frozen_inference_graph'

    # __init__()에서 모델 구조를 직접 구현하는 대신, tar file에서 읽어들인 그래프구조 graph_def를 
    # tf.compat.v1.import_graph_def를 통해 불러들여 활용하게 됩니다. 
    def __init__(self, tarball_path):
        self.graph = tf.Graph()
        graph_def = None
        tar_file = tarfile.open(tarball_path)
        for tar_info in tar_file.getmembers():
            if self.FROZEN_GRAPH_NAME in os.path.basename(tar_info.name):
                file_handle = tar_file.extractfile(tar_info)
                graph_def = tf.compat.v1.GraphDef.FromString(file_handle.read())
                break
        tar_file.close()

        with self.graph.as_default():
    	    tf.compat.v1.import_graph_def(graph_def, name='')

        self.sess = tf.compat.v1.Session(graph=self.graph)

    # 이미지를 전처리하여 Tensorflow 입력으로 사용 가능한 shape의 Numpy Array로 변환합니다.
    def preprocess(self, img_orig):
        height, width = img_orig.shape[:2]
        resize_ratio = 1.0 * self.INPUT_SIZE / max(width, height)
        target_size = (int(resize_ratio * width), int(resize_ratio * height))
        resized_image = cv2.resize(img_orig, target_size)
        resized_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
        img_input = resized_rgb
        return img_input
        
    def run(self, image):
        img_input = self.preprocess(image)

        # Tensorflow V1에서는 model(input) 방식이 아니라 sess.run(feed_dict={input...}) 방식을 활용합니다.
        batch_seg_map = self.sess.run(
            self.OUTPUT_TENSOR_NAME,
            feed_dict={self.INPUT_TENSOR_NAME: [img_input]})

        seg_map = batch_seg_map[0]
        return cv2.cvtColor(img_input, cv2.COLOR_RGB2BGR), seg_map
    
_DOWNLOAD_URL_PREFIX = 'http://download.tensorflow.org/models/'

model_dir = '~/aiffel/human_segmentation/models'
tf.io.gfile.makedirs(model_dir)

download_path = os.path.join(model_dir, 'deeplab_model.tar.gz')
if not os.path.exists(download_path):
    urllib.request.urlretrieve(_DOWNLOAD_URL_PREFIX + 'deeplabv3_mnv2_pascal_train_aug_2018_01_29.tar.gz',
                   download_path)

MODEL = DeepLabModel(download_path)

In [ ]:
#img_path = os.getenv('HOME')+'/aiffel/human_segmentation/images/cat1.PNG'  # 본인이 선택한 이미지의 경로에 맞게 바꿔 주세요. 
img_path = 'cat3.PNG'
img_orig = cv2.imread(img_path) 
plt.imshow(img_orig)

In [ ]:
img_resized = None
seg_map = None
img_resized, seg_map = MODEL.run(img_orig)
seg_map_origin = np.where(seg_map == 8, 8, 0)


In [ ]:
seg_map.max()

In [ ]:
img_show = img_resized.copy()
seg_map = np.where(seg_map == 8, 8, 0) # 예측 중 사람만 추출
img_mask = seg_map * (255/seg_map.max()) # 255 normalization
img_mask = img_mask.astype(np.uint8)
color_mask = cv2.applyColorMap(img_mask, cv2.COLORMAP_JET)

img_mask_up = cv2.resize(img_mask, img_orig.shape[:2][::-1], interpolation=cv2.INTER_LINEAR)
_, img_mask_up = cv2.threshold(img_mask_up, 128, 255, cv2.THRESH_BINARY)

img_mask_color = cv2.cvtColor(img_mask_up, cv2.COLOR_GRAY2BGR)
img_bg_mask = cv2.bitwise_not(img_mask_color)
img_bg = cv2.bitwise_and(img_orig, img_bg_mask)
img_bg_blur = cv2.blur(img_bg, (3,3))
img_concat = np.where(img_mask_color==255, img_orig,img_bg_blur)

plt.imshow(cv2.cvtColor(img_concat, cv2.COLOR_BGR2RGB))


In [ ]:
img=cv2.cvtColor(img_concat, cv2.COLOR_BGR2RGB)
from PIL import Image
img = Image.fromarray(img)
img.save("cat3_result.png")

# 해결방안 제시해 보기

 1. 문제점 : 위의 결과 파일에서 보는 것과 같이, 머리카락, 동물의 꼬리와 같은 부분은 제대로 분류하지 못하는 경우도 생겨난다. 따라서 Segmentation을 이용하는 방법은 적절하지 못하다.
 
 2. 원인 : Segmentation의 분류 영역이 100% 일치하지 못하며, 피사계심도를 구분하지 못해 정확한 거리를 통한 Out Focusing기능이 적용되지 못한다.
 
 3. 해결책 : Depth 정보를 이용한다.
 
 #### Depth Image
    
    
  - Stereo type : 카메라 2 대를 사용해서 Depth 정보를 추출해낼 수 있다. 하지만 카메라가 좌우로 수평하게 놓여있어야 Depth 정보가 추출될 수 있다. 휴대폰의 카메라를 수평으로 고정한 상태에서 사진을 찍기 어려우므로 실질적으로 가능하지 못하다.
   [참고]https://dev.intelrealsense.com/docs/stereo-depth-cameras-for-phones   
   
   
   
   - ToF type : IR 와 같은 주파수를 내보내고 돌아오는 시간을 계산하여 Depth 정보를 추출한다. 카메라가 비싸지만 Mobile에서는 ToF방식이 적절할것 같다. 따라서  ToF 방식 선택
   
   
   

![img](shallow_focusing_Diagram.png)